In [ ]:
#!/usr/bin/env python
def generate_url(year):
    
    #generate the url for fetching the log files for every month's first day
    number_of_months=1
    while number_of_months < 13:
        if(number_of_months <10):
            url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+'%02d' % number_of_months+"01.zip"
        else:
            url="http://www.sec.gov/dera/data/PublicEDGAR-log-file-data/"+year+"/Qtr1/log"+year+str(number_of_months)+"01.zip"
        number_of_months=number_of_months+1
    temp_url=download_data("http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip")
    download_data(temp_url)
    
def download_data(url):
    print("In download URL function")
    #you need to install request package using $ pip install requests
    from io import BytesIO
    from zipfile import ZipFile
    import urllib.request
    import csv
    url = urllib.request.urlopen(url)
       
    with ZipFile(BytesIO(url.read())) as my_zip_file:
        for contained_file in my_zip_file.namelist():
            print("unzipping maybe")
            uncompress_size = sum((file.file_size for file in my_zip_file.infolist()))

            extracted_size = 0

            for file in my_zip_file.infolist():
                extracted_size += file.file_size
                print (extracted_size * 100/uncompress_size)
                my_zip_file.extract(file)
                
            # with open(("unzipped_and_read_" + contained_file + ".file"), "wb") as output:
            
            for line in my_zip_file.open(contained_file).readlines():
                print("reading line one by one started")
                line=[line]
                with open('test.csv', 'w', newline='') as fp:
                    a = csv.writer(fp, delimiter=',')
                    a.writerows(line)
            print("Done")

    
if __name__ == '__main__':
    #fetch the year for which the user wants logs
    year = input('Enter the year for which you need to fetch the log files: ')
    #calling the function to generate dynamic URL
    generate_url(year)